# Data loading

In [1]:
import torch

In [2]:
# Russian tokenizer
!python -m spacy download ru_core_news_sm
import spacy
nlp = spacy.load("ru_core_news_sm")

2023-07-03 07:20:38.180949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 07:20:39.236510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 80.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=0afb32dedc741f19536001fdf1ad0483f610f43fcff65746e7784252403dcd7f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built doc

In [2]:
# Download dataset
!wget https://raw.githubusercontent.com/senyakk/Gogol-GPT/main/gogol.txt

--2023-07-03 09:08:40--  https://raw.githubusercontent.com/senyakk/Gogol-GPT/main/gogol.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 342790 (335K) [text/plain]
Saving to: ‘gogol.txt’

gogol.txt           100%[===================>] 334.76K  --.-KB/s    in 0.01s   

2023-07-03 09:08:40 (23.5 MB/s) - ‘gogol.txt’ saved [342790/342790]



# Data preprocessing

In [3]:
# Read dataset
with open('gogol.txt', 'r', encoding='cp1251') as file:
  text = file.read()

In [6]:
# CHARACTER VERSION

# here are all the unique characters that occur in this text
vocabulary = sorted(list(set(text)))
vocab_size = len(vocabulary)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocabulary) }
itos = { i:ch for i,ch in enumerate(vocabulary) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string



In [6]:
# TOKEN VERSION

#Tokenize the text
doc = nlp(text)

# Create a vocabulary set
vocabulary = set(token.text for token in doc)
ttoi = {t: i for i, t in enumerate(vocabulary)} # Mapping of vocabulary
itos = {i: t for t, i in ttoi.items()}  # Reverse mapping of vocabulary

encode = lambda s: [ttoi[t] for t in s]  # encoder: take a string, output a list of integers
decode = lambda l: ' '.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

test = "идет мужик по городу\n"
tokens = [token.text for token in nlp(test)]
encoded_text = encode(tokens)
decoded_text = decode([567, 2345, 1097, 698])

print(encoded_text)
print(decoded_text)

[5581, 5473, 9352, 10237, 899]
сотен невидаль из дирекция


In [7]:
vocab_size = len(vocabulary)
print(vocab_size)

113


In [9]:
# Encode dataset to tokens and store it into a torch Tensor

 # data = torch.tensor(encode(token.text for token in doc), dtype=torch.long)

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:50]) # the 50 characters

torch.Size([340157]) torch.int64
tensor([  1,  61,   9,  50,   9,   2,  51,  91,  80,  91,  88, 105,   9,   2,
         64,  82,  79,  85,  84,  91,  93,   1,   1,  61,  85,  87,  91,  88,
         77,  86,   2,  50,  77,  94,  85,  88, 105,  82,  79,  85, 100,   2,
         51,  91,  80,  91,  88, 105,   1,   1])


# Data split

In [10]:
# Split to training and validation sets
n = int (0.9 * len(data)) # First 90% is the training set
train_data = data[:n]
val_data = data[n:]

# Building Model

In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 16
block_size = 32
max_iters = 10000
eval_interval = 200
learning_rate = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.0

In [12]:
torch.manual_seed(2552)

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [13]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range (eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train
  return out

In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [15]:
torch.manual_seed(2552)

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
model = model.to(device)

In [17]:
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

0.058865 M parameters


In [18]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training

In [ ]:
for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


step 0: train loss 1.8947, val loss 2.039706
step 200: train loss 1.8832, val loss 2.039758
step 400: train loss 1.8892, val loss 2.051541
step 600: train loss 1.8604, val loss 2.006713
step 800: train loss 1.8379, val loss 2.016460
step 1000: train loss 1.8589, val loss 2.006423
step 1200: train loss 1.8409, val loss 2.009934
step 1400: train loss 1.8190, val loss 2.005933
step 1600: train loss 1.8257, val loss 1.993416
step 1800: train loss 1.8042, val loss 2.006117
step 2000: train loss 1.7954, val loss 1.976690
step 2200: train loss 1.8156, val loss 1.990073
step 2400: train loss 1.7790, val loss 1.957048
step 2600: train loss 1.8084, val loss 1.992993
step 2800: train loss 1.7881, val loss 1.964431
step 3000: train loss 1.7790, val loss 1.960931
step 3200: train loss 1.8046, val loss 1.963025
step 3400: train loss 1.7805, val loss 1.962472
step 3600: train loss 1.7596, val loss 1.935198
step 3800: train loss 1.7804, val loss 1.964097
step 4000: train loss 1.7772, val loss 1.981235

# Generating

In [21]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

	тованна коле… А ! что, с вес тыто, что, коним и — его


Александович. Много мне сне, шлет, подеетерет и ухухт гогородничий скаходин. Я он боль, бвишка, коне Итанкуми гля, каль кра о и мет. — посутая тебе блушку.. Дылогодой бремезнею чиза в ему. йде-то не да хочертнось сгородке закольновадом ми выне оему и пости? послеги с метелужны только с соке!»» Раднысь. акрайтесь потисть! (Клелаще ссходир еменятихесадит на чета? Иге правек, — влочест.. Хлев лас большку лего.(кутивед руку и утакойенниа мичего
